**How many table want to generate?**

In [1]:
number_of_table = 50

In [10]:
!pip install reportlab
!pip install pdf2image
!apt-get update
!apt-get install poppler-utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 20 not upgraded.
Need to get 154 kB of archives.
After this operation, 613 kB of additional disk space will be used.
Ign:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 poppler-utils amd64 0.62.0-2ubuntu2.13
Err:1 http://security.ubuntu.com/ubuntu bionic-updates/main amd64 poppler-utils amd64 0.62.0-2ubuntu2.13
  404  Not Found [IP: 91.189.91.39 80]
E: Failed to fetch http://security.ubuntu.com/ubuntu/pool/main/p/poppler/poppler-utils_0.62.0-2ub

In [11]:
# https://icdlist.com/
!git clone https://github.com/MZulfadzli/geninvoicetable.git
!unzip -qq -o geninvoicetable/all_data.zip -d ./
!mv ./geninvoicetable/* ./
!rm -r ./geninvoicetable

import os
import random
import json

myjsonfile = open('druglist.json','r')
jsondata = myjsonfile.read()
drug_list = json.loads(jsondata)

myjsonfile = open('servicelist.json','r')
jsondata = myjsonfile.read()
services_list = json.loads(jsondata)

myjsonfile = open('equiplist.json','r')
jsondata = myjsonfile.read()
equip_list = json.loads(jsondata)

Cloning into 'geninvoicetable'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 42 (delta 11), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (42/42), done.


In [12]:
try:
    if not os.path.exists('generated_table'):
        os.makedirs('generated_table')
except OSError:
    print ('Error: Creating directory of in_images')

In [5]:
from reportlab.platypus import SimpleDocTemplate, Paragraph
from reportlab.lib.pagesizes import A4
from reportlab.lib.styles import ParagraphStyle
from reportlab.platypus import Table, TableStyle
from reportlab.lib import colors
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from pdf2image import convert_from_path, convert_from_bytes
from IPython.display import display, Image
import numpy as np
import cv2

filename = 'SyntheticTable.pdf'

In [6]:
# register font from ttf file

# Roboto
pdfmetrics.registerFont(TTFont('Roboto', 'Roboto-Light.ttf'))
pdfmetrics.registerFont(TTFont('Roboto-Bold', 'Roboto-Bold.ttf'))

# Arial
pdfmetrics.registerFont(TTFont('Arial', 'ARIAL.TTF'))
pdfmetrics.registerFont(TTFont('Arial-Bold', 'ARIALBD.TTF'))

# Calibri
pdfmetrics.registerFont(TTFont('Calibri', 'CALIBRI.TTF'))
pdfmetrics.registerFont(TTFont('Calibri-Bold', 'CALIBRIB.TTF'))

In [7]:
def rotate_image(image, angle):
  image_center = tuple(np.array(image.shape[1::-1]) / 2)
  rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
  result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR, borderValue=(255,255,255))
  return result

def sp_noise(image,prob):
    
    #Add salt and pepper noise to image | prob: Probability of the noise
    output = np.zeros(image.shape,np.uint8)
    thres = 1 - prob 
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            rdn = random.random()
            if rdn < prob:
                output[i][j] = 0
            elif rdn > thres:
                output[i][j] = 255
            else:
                output[i][j] = image[i][j]
    return output

In [8]:
def generate_table():
  # Randomize type of surgery
  currentdir = 'all_data'
  choice1 = os.listdir(currentdir)
  addpath=random.choice(choice1)
  currentdir = 'all_data/'+addpath
  if os.path.isdir(currentdir):  
      #print("\nIt is a directory")
      choice2 = os.listdir(currentdir)
      addpath2=random.choice(choice2)
      currentdir = 'all_data/'+addpath+'/'+addpath2
      if os.path.isdir(currentdir):
        #print("\nIt is a directory")
        choice3 = os.listdir(currentdir)
        addpath3=random.choice(choice3)
        currentdir = 'all_data/'+addpath+'/'+addpath2+'/'+addpath3
      elif os.path.isfile(currentdir):
        #print("\nIt is a normal file")
        currentdir = currentdir
      else:
        print("It is a special file (socket, FIFO, device file)" )
  elif os.path.isfile(currentdir):  
      #print("\nIt is a normal file")
      currentdir = currentdir  
  else:  
      print("It is a special file (socket, FIFO, device file)" )

  #print(currentdir)
  myjsonfile = open(currentdir,'r')
  jsondata = myjsonfile.read()
  medication_list = json.loads(jsondata)

  # Randomize font
  font_list = ['Helvetica', 'Arial','Courier', 'Roboto', 'Calibri']
  myFont = random.choice(font_list)
  myFontB = myFont + "-Bold"

  # Randomize font size
  font_size = random.randint(9,11)

  # Randomize color
  list_color = [colors.black, colors.white]
  selected_col = random.choice(list_color)

  # Randomize border line
  b_line = 0 if selected_col == colors.white else random.randint(0,2)

  # Randomize background total
  list_totbackg = [colors.lightgrey, colors.beige, colors.aqua]
  selected_totbackg = random.choice(list_totbackg)

  # Randomize padding
  list_padding = [0, 1, 2, 3]
  selected_padding = random.choice(list_padding)

  # Randomize layout
  layout = [0,1]
  logic = random.choice(layout)

  # Randomize left margin
  left = random.randint(10,40)

  # Randomize right margin
  right = random.randint(10,40)

  # Randomize top margin
  top = random.randint(30,350)

  # Randomize bottom margin
  bottom = random.randint(5,30)

  # Create new paragraph style
  myStyle = ParagraphStyle(
      'mystyle',
      fontName=myFont,
      fontSize=font_size,
      )
  if logic == 1:
    datatotable = [["Item Code","Description", "Qty", "Amount", "Disc.Amount", "Total"],["", "", "", "(RM)", "(RM)", "(RM)"], ["SURGERY CHARGES"]]
    style = TableStyle([
      ('TEXTCOLOR', (0,0), (-1,-1), colors.black),
      ('FONTNAME', (0,2), (-1,2), myFontB),
      ('FONTNAME', (0,0), (-1,1), myFont),
      ('FONTNAME', (0,3), (-1,-1), myFont),
      ('FONTSIZE', (0,0), (-1,-1), font_size),
      ('LINEABOVE', (0,0), (-1,0), b_line, selected_col),
      ('LINEBELOW', (0,1), (-1,1), b_line, selected_col),
      ('TOPPADDING', (0,0), (-1,1), selected_padding),
      ('BOTTOMPADDING', (0,0), (-1,1), selected_padding),
      ('ALIGN', (0,1), (-1,1), 'RIGHT'),
      ('ALIGN', (2,0), (-1,-1), 'RIGHT'),
      ('ALIGN', (2,0), (2,-1), 'CENTER'),
      ('VALIGN', (0,0), (-1,-1), 'TOP')
    ])
    adddatatotable2 = [["EQUIPMENT CHARGES"]]
    style4 = TableStyle([
      ('TEXTCOLOR', (0,0), (-1,-1), colors.black),
      ('FONTNAME', (0,0), (-1,0), myFontB),
      ('FONTNAME', (0,1), (-1,-1), myFont),
      ('FONTSIZE', (0,0), (-1,-1), font_size),
      ('TOPPADDING', (0,0), (-1,-1), selected_padding),
      ('BOTTOMPADDING', (0,0), (-1,-1), selected_padding),
      ('ALIGN', (3,0), (-1,-1), 'RIGHT'),
      ('ALIGN', (2,0), (2,-1), 'CENTER'),
      ('VALIGN', (0,0), (-1,-1), 'TOP')
    ])
    adddatatotable3 = [["HOSPITALITY"]]
    style5 = style4
    adddatatotable = [["MEDICINE CHARGES"]]
    style2 = style4
  else:
    datatotable = [["Item Code","Description", "Qty", "Amount", "Disc.Amount", "Total"],["", "", "", "(RM)", "(RM)", "(RM)"]]
    style = TableStyle([
      ('TEXTCOLOR', (0,0), (-1,-1), colors.black),
      ('FONTNAME', (0,0), (-1,-1), myFont),
      ('FONTSIZE', (0,0), (-1,-1), font_size),
      ('LINEABOVE', (0,0), (-1,0), b_line, selected_col),
      ('LINEBELOW', (0,1), (-1,1), b_line, selected_col),
      ('TOPPADDING', (0,0), (-1,1), selected_padding),
      ('BOTTOMPADDING', (0,0), (-1,1), selected_padding),
      ('ALIGN', (0,1), (-1,1), 'RIGHT'),
      ('ALIGN', (2,0), (-1,-1), 'RIGHT'),
      ('ALIGN', (2,0), (2,-1), 'CENTER'),
      ('VALIGN', (0,0), (-1,-1), 'TOP')
    ])
    adddatatotable2 = []
    style4 = TableStyle([
      ('TEXTCOLOR', (0,0), (-1,-1), colors.black),
      ('FONTNAME', (0,0), (-1,-1), myFont),
      ('FONTSIZE', (0,0), (-1,-1), font_size),
      ('TOPPADDING', (0,0), (-1,-1), selected_padding),
      ('BOTTOMPADDING', (0,0), (-1,-1), selected_padding),
      ('ALIGN', (3,0), (-1,-1), 'RIGHT'),
      ('ALIGN', (2,0), (2,-1), 'CENTER'),
      ('VALIGN', (0,0), (-1,-1), 'TOP')
    ])
    adddatatotable3 = []
    style5 = style4
    adddatatotable = []
    style2 = style4

  l=list(medication_list.keys())
  sumtotal = 0
  number_list = 1
  a = random.sample(range(1,len(medication_list)), number_list)

  for i in a:
    code = l[i]
    pres = medication_list[l[i]]
    qty = 1
    disc = 0
    amount = round(random.uniform(5000,9999), 2)
    total = qty*amount
    sumtotal = sumtotal + total
    amountdum = "{:.2f}".format(amount)
    totaldum = "{:.2f}".format(total)
    datatotable.append([Paragraph(code, myStyle), Paragraph(pres, myStyle), qty, amountdum, disc, totaldum])

  table = Table(datatotable, colWidths=[80, 200, 40, 70, 100, 70])

  table.setStyle(style)
  elem = []
  elem.append(table)
#
  l3=list(equip_list.keys())
  number_list3 = random.randint(4,7)
  b = random.sample(range(0,len(equip_list)), number_list3)

  for i in b:
    code = l3[i]
    pres = equip_list[l3[i]]
    qty = 1
    disc = 0
    amount = round(random.uniform(50,100), 2)
    total = qty*amount
    sumtotal = sumtotal + total
    amountdum = "{:.2f}".format(amount)
    totaldum = "{:.2f}".format(total)
    adddatatotable2.append([Paragraph(code, myStyle), Paragraph(pres, myStyle), qty, amountdum, disc, totaldum])

  table4 = Table(adddatatotable2, colWidths=[80, 200, 40, 70, 100, 70])

  table4.setStyle(style4)
  elem.append(table4)
# 
  l4=list(services_list.keys())
  number_list4 = random.randint(3,4)
  b = random.sample(range(0,len(services_list)), number_list4)

  for i in b:
    code = l4[i]
    pres = services_list[l4[i]]
    if i==1:
      qty = random.randint(2,6)
      amount = round(69.99)
    else:
      qty = 1
      amount = round(random.uniform(100,300), 2)
    
    
    disc = 0
    total = qty*amount
    sumtotal = sumtotal + total
    amountdum = "{:.2f}".format(amount)
    totaldum = "{:.2f}".format(total)
    adddatatotable3.append([Paragraph(code, myStyle), Paragraph(pres, myStyle), qty, amountdum, disc, totaldum])

  table5 = Table(adddatatotable3, colWidths=[80, 200, 40, 70, 100, 70])

  table5.setStyle(style5)
  elem.append(table5)
# 
  l2=list(drug_list.keys())
  number_list2 = random.randint(2,5)
  b = random.sample(range(0,len(drug_list)), number_list2)

  for i in b:
    code = l2[i]
    pres = drug_list[l2[i]]
    qty = random.randint(1,4)
    disc = 0
    amount = round(random.uniform(20,50), 2)
    total = qty*amount
    sumtotal = sumtotal + total
    amountdum = "{:.2f}".format(amount)
    totaldum = "{:.2f}".format(total)
    adddatatotable.append([Paragraph(code, myStyle), Paragraph(pres, myStyle), qty, amountdum, disc, totaldum])

  table2 = Table(adddatatotable, colWidths=[80, 200, 40, 70, 100, 70])

  table2.setStyle(style2)
  elem.append(table2)
#
  sumtotal = round(sumtotal, 2)
  tax = round(0.06*sumtotal, 2)
  finaltotal = sumtotal + tax
  sumtotaldum = "{:.2f}".format(sumtotal)
  taxdum = "{:.2f}".format(tax)
  finaltotaldum = "{:.2f}".format(finaltotal)
  finaldata = [["CHARGES", sumtotaldum], ["TAX(6%)", taxdum], ["TOTAL CHARGES", finaltotaldum]]
  table3 = Table(finaldata, colWidths=[490, 70])
  style3 = TableStyle([
      ('TEXTCOLOR', (0,0), (-1,-1), colors.black),
      ('FONTNAME', (0,0), (-1,1), myFont),
      ('FONTNAME', (0,2), (-1,2), myFontB),
      ('FONTSIZE', (0,0), (-1,-1), font_size),
      ('LINEABOVE', (0,0), (-1,0), b_line, selected_col),
      ('LINEBELOW', (0,-1), (-1,-1), b_line, selected_col),
      ('TOPPADDING', (0,0), (-1,-1), selected_padding),
      ('BOTTOMPADDING', (0,0), (-1,-1), selected_padding),
      ('ALIGN', (1,0), (-1,-1), 'RIGHT'),
      ('VALIGN', (0,0), (-1,-1), 'TOP'),
      ('BACKGROUND', (0,-1), (-1,-1), selected_totbackg)
  ])
  table3.setStyle(style3)
  elem.append(table3)
  pdf = SimpleDocTemplate(filename=filename, pagesize=A4, rightMargin=right, leftMargin=left, topMargin=top, bottomMargin=bottom)
  pdf.build(elem)

  images = convert_from_bytes(open('SyntheticTable.pdf', 'rb').read(), size=1400)
  #display(images[0])
  images[0].save('pdftoimage.png')

  image = cv2.imread('pdftoimage.png',3) # 1 Only for grayscale image

  # Random rotation
  DEGREE = random.uniform(-3,3)
  #print(DEGREE)
  rotated_img = rotate_image(image, DEGREE)
  p = [0.01, 0.02, 0.03]
  p_noise = random.choice(p)
  noise_img = sp_noise(rotated_img,p_noise)

  return noise_img

In [17]:
for i in range(number_of_table):
  final_image = generate_table()
  cv2.imwrite('generated_table/gentable'+str(i)+'.jpg', final_image)

In [18]:
!zip -r -o -qq generated_table.zip ./generated_table/

In [19]:
from google.colab import files
files.download('generated_table.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
#!dpkg -L poppler-utils

/.
/usr
/usr/bin
/usr/bin/pdfdetach
/usr/bin/pdffonts
/usr/bin/pdfimages
/usr/bin/pdfinfo
/usr/bin/pdfseparate
/usr/bin/pdfsig
/usr/bin/pdftocairo
/usr/bin/pdftohtml
/usr/bin/pdftoppm
/usr/bin/pdftops
/usr/bin/pdftotext
/usr/bin/pdfunite
/usr/share
/usr/share/doc
/usr/share/doc/poppler-utils
/usr/share/doc/poppler-utils/copyright
/usr/share/lintian
/usr/share/lintian/overrides
/usr/share/lintian/overrides/poppler-utils
/usr/share/man
/usr/share/man/man1
/usr/share/man/man1/pdfdetach.1.gz
/usr/share/man/man1/pdffonts.1.gz
/usr/share/man/man1/pdfimages.1.gz
/usr/share/man/man1/pdfinfo.1.gz
/usr/share/man/man1/pdfseparate.1.gz
/usr/share/man/man1/pdfsig.1.gz
/usr/share/man/man1/pdftocairo.1.gz
/usr/share/man/man1/pdftohtml.1.gz
/usr/share/man/man1/pdftoppm.1.gz
/usr/share/man/man1/pdftops.1.gz
/usr/share/man/man1/pdftotext.1.gz
/usr/share/man/man1/pdfunite.1.gz
/usr/share/doc/poppler-utils/changelog.Debian.gz


In [15]:
#!apt-get install poppler-utils

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libpoppler-dev libpoppler-private-dev libpoppler73
Recommended packages:
  poppler-data
The following NEW packages will be installed:
  poppler-utils
The following packages will be upgraded:
  libpoppler-dev libpoppler-private-dev libpoppler73
3 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 1,128 kB of archives.
After this operation, 614 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libpoppler-private-dev amd64 0.62.0-2ubuntu2.14 [169 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libpoppler-dev amd64 0.62.0-2ubuntu2.14 [4,616 B]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 li